# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [1]:
import numpy as np
import math
import openmc
import openmc.deplete as od
from openmcyclus.depletion import Depletion

In [2]:
uox_model = openmc.Model()
mox_model = openmc.Model()
openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [3]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 0.4778 # area of circle for fuel

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 0.4778 # area of circle for fuel

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
mox_model.Materials = openmc.Materials([mox, zirconium, water])

In [4]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

uox_fuel = openmc.Cell(name='fuel')
uox_fuel.fill = uo2
uox_fuel.region = fuel_region

mox_fuel = openmc.Cell(name='fuel')
mox_fuel.fill = mox
mox_fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
box = openmc.rectangular_prism(width=pitch, height=pitch,
                               boundary_type='reflective')
water_region = box & +clad_outer_radius
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

uox_universe = openmc.Universe(cells=(uox_fuel, gap, clad, moderator))
mox_universe = openmc.Universe(cells=(mox_fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(uox_universe)
mox_model.geometry = openmc.Geometry(mox_universe)

In [5]:
# Settings
settings = openmc.Settings()
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings

In [6]:
# Generate one-group cross section data
uox_cross_sections = od.MicroXS.from_model(uox_model, 
                                       domain = uox_fuel, 
                                       chain_file="chain_endfb71_pwr.xml")
mox_cross_sections = od.MicroXS.from_model(mox_model, 
                                       domain = mox_fuel, 
                                       chain_file="chain_endfb71_pwr.xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Reading Ca42 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca42.h5
 Reading Ca43 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca43.h5
 Reading Ca44 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca44.h5
 Reading Ca45 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca45.h5
 Reading Ca46 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca46.h5
 Reading Ca47 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca47.h5
 Reading Ca48 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ca48.h5
 Reading Sc45 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sc45.h5
 Reading Ti46 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ti46.h5
 Reading Ti47 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ti47.h5
 Reading Ti48 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.

 Reading Sr85 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr85.h5
 Reading Sr86 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr86.h5
 Reading Sr87 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr87.h5
 Reading Sr88 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr88.h5
 Reading Sr89 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr89.h5
 Reading Sr90 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sr90.h5
 Reading Y89 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Y89.h5
 Reading Y90 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Y90.h5
 Reading Y91 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Y91.h5
 Reading Zr93 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Zr93.h5
 Reading Zr95 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5

 Reading Cd110 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd110.h5
 Reading Cd111 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd111.h5
 Reading Cd112 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd112.h5
 Reading Cd113 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd113.h5
 Reading Cd114 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd114.h5
 Reading Cd115_m1 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd115_m1.h5
 Reading Cd116 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Cd116.h5
 Reading In113 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/In113.h5
 Reading In114 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/In114.h5
 Reading In115 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/In115.h5
 Reading Sn112 from
 /home/abachmann@anl.gov

 Reading Ba132 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba132.h5
 Reading Ba133 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba133.h5
 Reading Ba134 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba134.h5
 Reading Ba135 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba135.h5
 Reading Ba136 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba136.h5
 Reading Ba137 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba137.h5
 Reading Ba138 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba138.h5
 Reading Ba139 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba139.h5
 Reading Ba140 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Ba140.h5
 Reading La138 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/La138.h5
 Reading La139 from
 /home/abachmann@anl.gov/openm

...

 Reading Te131_m1 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Te131_m1.h5
 Reading Te132 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Te132.h5
 Reading I127 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I127.h5
 Reading I128 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I128.h5
 Reading I129 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I129.h5
 Reading I130 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I130.h5
 Reading I131 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I131.h5
 Reading I132 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I132.h5
 Reading I132_m1 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I132_m1.h5
 Reading I133 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I133.h5
 Reading I134 from
 /home/abachmann@anl.gov/openmc-xs-

 Reading Sm147 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm147.h5
 Reading Sm148 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm148.h5
 Reading Sm149 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm149.h5
 Reading Sm150 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm150.h5
 Reading Sm151 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm151.h5
 Reading Sm152 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm152.h5
 Reading Sm153 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm153.h5
 Reading Sm154 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sm154.h5
 Reading Eu151 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Eu151.h5
 Reading Eu152 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Eu152.h5
 Reading Eu153 from
 /home/abachmann@anl.gov/openm

 Reading Yb176 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Yb176.h5
          1200K
          2500K
 Reading Lu175 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Lu175.h5
 Reading Lu176 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Lu176.h5
 Reading Hf174 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf174.h5
 Reading Hf175 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf175.h5
 Reading Hf176 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf176.h5
 Reading Hf177 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf177.h5
 Reading Hf178 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf178.h5
 Reading Hf179 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf179.h5
 Reading Hf180 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Hf180.h5
 Reading Hf181 fro

 Reading Th230 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Th230.h5
 Reading Th231 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Th231.h5
 Reading Th232 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Th232.h5
 Reading Th233 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Th233.h5
 Reading Th234 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Th234.h5
 Reading Pa229 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Pa229.h5
 Reading Pa230 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Pa230.h5
 Reading Pa231 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Pa231.h5
 Reading Pa232 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Pa232.h5
 Reading Pa233 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Pa233.h5
 Reading U230 from
 /home/abachmann@anl.gov/openmc

       14/1    1.33344    1.45105 +/- 0.04044
       15/1    1.49112    1.45906 +/- 0.03233
       16/1    1.41654    1.45197 +/- 0.02734
       17/1    1.42500    1.44812 +/- 0.02342
       18/1    1.43183    1.44609 +/- 0.02039
       19/1    1.51697    1.45396 +/- 0.01963
       20/1    1.44299    1.45286 +/- 0.01759
       21/1    1.41707    1.44961 +/- 0.01624
       22/1    1.51200    1.45481 +/- 0.01571
       23/1    1.36346    1.44778 +/- 0.01607
       24/1    1.41027    1.44510 +/- 0.01512
       25/1    1.48142    1.44752 +/- 0.01428
       26/1    1.50225    1.45094 +/- 0.01379
       27/1    1.57285    1.45812 +/- 0.01480
       28/1    1.46403    1.45844 +/- 0.01396
       29/1    1.50191    1.46073 +/- 0.01340
       30/1    1.48303    1.46185 +/- 0.01276
       31/1    1.39397    1.45861 +/- 0.01256
       32/1    1.50496    1.46072 +/- 0.01216
       33/1    1.44608    1.46008 +/- 0.01164
       34/1    1.48588    1.46116 +/- 0.01120
       35/1    1.43803    1.46023 

In [7]:
# save cross sections from UOX model for use in Deplete Reactor
uox_cross_sections.to_csv("micro_xs.csv")
mox_cross_sections.to_csv("mox_xs.csv")

In [8]:
uox_cross_sections = od.MicroXS.from_csv("micro_xs.csv")
mox_cross_sections = od.MicroXS.from_csv("mox_xs.csv")

In [9]:
uox_ind_op = od.IndependentOperator(uox_model.Materials, 
                                uox_cross_sections, 
                                "chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(mox_model.Materials, 
                                mox_cross_sections, 
                                "chain_endfb71_pwr.xml")

In [10]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    np.ones(12*3)*30, # 3 cycles of operation
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=33696000.0 s, dt

In [11]:
! mv depletion_results.h5 uox_depletion_results.h5

In [12]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12*3)*30,
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=33696000.0 s, dt

In [13]:
! mv depletion_results.h5 mox_depletion_results.h5

In [14]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [15]:
uox_spent = uox_results.export_to_materials(-1)
mox_spent = mox_results.export_to_materials(-1)

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)


In [18]:
#printing in format for Cyclus recipe
total = 0
weight_frac = 0
pu_frac = 0
for nuclide in uox_spent[0].nuclides:
    total += nuclide.percent
    if nuclide.percent <= 1e-10:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")
    weight_frac += nuclide.percent*A
    if Z == 94:
        pu_frac += nuclide.percent*A
    total += nuclide.percent

<nuclide>  <id>471090000</id> <comp>6.53835661494617e-06</comp>  </nuclide>
<nuclide>  <id>471100001</id> <comp>1.2812731748596007e-07</comp>  </nuclide>
<nuclide>  <id>471110000</id> <comp>3.947609947030497e-08</comp>  </nuclide>
<nuclide>  <id>471120000</id> <comp>2.9218759964888387e-10</comp>  </nuclide>
<nuclide>  <id>471130000</id> <comp>2.1831898777518225e-10</comp>  </nuclide>
<nuclide>  <id>952410000</id> <comp>5.367500649806784e-07</comp>  </nuclide>
<nuclide>  <id>952420000</id> <comp>3.6347426667747755e-09</comp>  </nuclide>
<nuclide>  <id>952420001</id> <comp>6.108615057325568e-09</comp>  </nuclide>
<nuclide>  <id>952430000</id> <comp>8.20856146415737e-06</comp>  </nuclide>
<nuclide>  <id>952440000</id> <comp>9.464359871019893e-10</comp>  </nuclide>
<nuclide>  <id>952440001</id> <comp>6.159752815902361e-10</comp>  </nuclide>
<nuclide>  <id>330750000</id> <comp>1.212856987607413e-08</comp>  </nuclide>
<nuclide>  <id>561340000</id> <comp>3.865023497306766e-06</comp>  </nuclid

In [19]:
#printing in format for Cyclus recipe
for nuclide in mox_spent[0].nuclides:
    if nuclide.percent <= 1e-10:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")

<nuclide>  <id>922380000</id> <comp>85.53765972559535</comp>  </nuclide>
<nuclide>  <id>922350000</id> <comp>2.612078957002141</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>11.850261317402508</comp>  </nuclide>
